Downloading datasets and libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Extracting the cityscapes archive

In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/MLDL_repo/Cityscapes.zip'
extract_path = '/content/dataset'

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")

Extraction complete!


Extracting GTA5 dataset

In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/MLDL_repo/GTA5.zip'
extract_path = '/content/dataset'

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")

Extraction complete!


Adding project directory to Python path for module imports

In [2]:
import sys
sys.path.append('/content/drive/MyDrive/MLDL_repo/step4')

Install the latest version of the fvcore Python Library

In [3]:
!pip install -U fvcore

main

##Training a domain adaptation model using GTA5 (source) and Cityscapes (target) datasets


The following code sets up a domain adaptation pipeline where a BiSeNet feature extractor and a discriminator are trained adversarially using GTA5 as the source dataset and Cityscapes as the target. It prepares the datasets with appropriate augmentations, loads them into DataLoaders, initializes models, optimizers, and loss functions, and performs multi-epoch training to learn domain-invariant features while maintaining strong source-domain performance, with support for multi-GPU acceleration.



In [ ]:
import os
import re

def find_latest_checkpoint(checkpoint_dir):
    if not os.path.exists(checkpoint_dir):
        print(f"Checkpoint directory {checkpoint_dir} does not exist.")
        return None

    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith("bisenet_epoch_") and f.endswith(".pt")]
    if not checkpoints:
        print(f"No checkpoints found in {checkpoint_dir}.")
        return None

    def extract_epoch(fname):
        match = re.search(r"bisenet_epoch_(\d+)\.pt", fname)
        return int(match.group(1)) if match else -1

    checkpoints.sort(key=lambda x: extract_epoch(x), reverse=True)
    latest = checkpoints[0]
    print(f"The latest checkpoint is: {latest}")
    return os.path.join(checkpoint_dir, latest)


In [4]:
import torch
from torchvision import transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2

#transformation for gta5 dataset
def get_gta5_transforms():
    return A.Compose(
        [
            A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05, p=0.5),
            A.GaussianBlur(blur_limit=(3, 5), sigma_limit=(0.1, 1.0), p=0.5),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.0, p=0.5),
            A.Resize(720, 1280),
            A.Normalize(mean=(0.485, 0.456, 0.406),
                        std=(0.229, 0.224, 0.225)),
            ToTensorV2()
        ],
        additional_targets={'mask': 'mask'}  #  ensures 'mask' is treated as mask
    )




#transformation for cityscape dataset
def get_cityscape_transform():
    return A.Compose([
        A.Resize(512, 1024),
        A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
], additional_targets={'mask': 'mask'})





In [ ]:
def main():
    import os
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import DataLoader
    from sklearn.model_selection import train_test_split
    from datasets_custom.gta5_aug import GTA5
    from datasets_custom.cityscapes_aug import CityScapes
    from models.bisenet.build_bisenet import BiSeNet
    from models.bisenet.discriminator import FCDiscriminator
    from train_ad import train_one_epoch

    # Paths
    gta5_root = '/content/dataset/GTA5'
    cityscape_root = '/content/dataset/Cityscapes/Cityspaces'
    checkpoint_dir = '/content/drive/MyDrive/MLDL_repo/step4/checkpoint4a'
    final_model_path = "/content/drive/MyDrive/MLDL_repo/step4/final4a.pth"

    # Make sure checkpoint directory exists
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Load datasets
    full_dataset = GTA5(root=gta5_root)
    indices = list(range(len(full_dataset)))
    train_indices, val_indices = train_test_split(indices, test_size=0.30, random_state=42)
    source_dataset = GTA5(root=gta5_root, transform=get_gta5_transforms())
    target_dataset = CityScapes(root=cityscape_root, split='train', transform=get_cityscape_transform())
    test_dataset = CityScapes(root=cityscape_root, split='val', transform=get_cityscape_transform())

    print(f"Train dataset size: {len(target_dataset)}")
    print(f"Test dataset size: {len(test_dataset)}")

    source_loader = DataLoader(source_dataset, batch_size=2, shuffle=True, num_workers=2)
    target_loader = DataLoader(target_dataset, batch_size=2, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=2, shuffle=True, num_workers=2)

    # Training setup
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    num_classes = 19
    base_lr = 2.5e-4
    batch_size = 2
    epochs = 50
    start_epoch = 0
    context_path = 'resnet18'
    lambda_adv = 0.001

    # Models
    feature_extr = BiSeNet(num_classes=num_classes, context_path=context_path)
    discriminator = FCDiscriminator(num_classes=num_classes)

    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs")
        feature_extr = nn.DataParallel(feature_extr)
        discriminator = nn.DataParallel(discriminator)

    feature_extr = feature_extr.to(device)
    discriminator = discriminator.to(device)

    # Optimizers
    optimizer_f = optim.Adam(feature_extr.parameters(), lr=base_lr)
    optimizer_d = optim.Adam(discriminator.parameters(), lr=base_lr)

    # Loss functions
    featureExtr_loss = nn.CrossEntropyLoss(ignore_index=255)
    adv_loss = nn.BCEWithLogitsLoss()

    # Load checkpoint if available
    latest_ckpt = find_latest_checkpoint(checkpoint_dir)
    if latest_ckpt:
        print(f"Restoring from checkpoint: {latest_ckpt}")
        checkpoint = torch.load(latest_ckpt, map_location=device)

        # Load model state dicts with DataParallel support
        if isinstance(feature_extr, nn.DataParallel):
            feature_extr.module.load_state_dict(checkpoint['feature_extr_state_dict'])
        else:
            feature_extr.load_state_dict(checkpoint['feature_extr_state_dict'])

        if isinstance(discriminator, nn.DataParallel):
            discriminator.module.load_state_dict(checkpoint['discriminator_state_dict'])
        else:
            discriminator.load_state_dict(checkpoint['discriminator_state_dict'])

        optimizer_f.load_state_dict(checkpoint['optimizer_f_state_dict'])
        optimizer_d.load_state_dict(checkpoint['optimizer_d_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        print(f"Resuming from epoch {start_epoch}")
    else:
        print("No checkpoint found. Starting from scratch.")

    # Training loop
    for epoch in range(start_epoch, epochs):
        train_one_epoch(
            feature_extr, discriminator,
            source_loader, target_loader,
            optimizer_f, optimizer_d,
            featureExtr_loss, adv_loss,
            base_lr, epoch, epochs,
            device, lambda_adv
        )

        # Save checkpoint
        ckpt_path = os.path.join(checkpoint_dir, f'bisenet_epoch_{epoch}.pt')
        torch.save({
            'epoch': epoch,
            'feature_extr_state_dict': feature_extr.module.state_dict() if isinstance(feature_extr, nn.DataParallel) else feature_extr.state_dict(),
            'discriminator_state_dict': discriminator.module.state_dict() if isinstance(discriminator, nn.DataParallel) else discriminator.state_dict(),
            'optimizer_f_state_dict': optimizer_f.state_dict(),
            'optimizer_d_state_dict': optimizer_d.state_dict()
        }, ckpt_path)
        print(f"Checkpoint saved at {ckpt_path}")

    # Save final model
    torch.save(
        feature_extr.module.state_dict() if isinstance(feature_extr, nn.DataParallel) else feature_extr.state_dict(),
        final_model_path
    )
    print(f"Final model saved at: {final_model_path}")


if __name__ == "__main__":
    main()


Loaded 1572 images for split: train
Loaded 500 images for split: val
Train dataset size: 1572
Test dataset size: 500
The latest checkpoint is: bisenet_epoch_0.pt
Restoring from checkpoint: /content/drive/MyDrive/ML&DL/Progetto/MLDL_2025_main/checkpoint4a/bisenet_epoch_0.pt
Resuming from epoch 1

Epoch 2/50


100%|██████████| 786/786 [03:37<00:00,  3.62it/s]


 Avg Seg Loss: 0.4425 | Adv Loss: 0.0020 | D Loss: 0.4903
Checkpoint saved at /content/drive/MyDrive/ML&DL/Progetto/MLDL_2025_main/checkpoint4a/bisenet_epoch_1.pt

Epoch 3/50


100%|██████████| 786/786 [03:37<00:00,  3.62it/s]


 Avg Seg Loss: 0.3818 | Adv Loss: 0.0021 | D Loss: 0.4859
Checkpoint saved at /content/drive/MyDrive/ML&DL/Progetto/MLDL_2025_main/checkpoint4a/bisenet_epoch_2.pt

Epoch 4/50


...

 Avg Seg Loss: 0.0950 | Adv Loss: 0.0031 | D Loss: 0.4107
Checkpoint saved at /content/drive/MyDrive/ML&DL/Progetto/MLDL_2025_main/checkpoint4a/bisenet_epoch_47.pt

Epoch 49/50


100%|██████████| 786/786 [03:37<00:00,  3.61it/s]


 Avg Seg Loss: 0.0953 | Adv Loss: 0.0031 | D Loss: 0.4110
Checkpoint saved at /content/drive/MyDrive/ML&DL/Progetto/MLDL_2025_main/checkpoint4a/bisenet_epoch_48.pt

Epoch 50/50


100%|██████████| 786/786 [03:37<00:00,  3.61it/s]


 Avg Seg Loss: 0.0950 | Adv Loss: 0.0031 | D Loss: 0.4088
Checkpoint saved at /content/drive/MyDrive/ML&DL/Progetto/MLDL_2025_main/checkpoint4a/bisenet_epoch_49.pt
Final model saved at: /content/drive/MyDrive/ML&DL/Progetto/MLDL_2025_main/final4a.pth


In [6]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from datasets_custom.cityscapes_aug import CityScapes
from models.bisenet.build_bisenet import BiSeNet
from train_ad import validate  # Assumes this returns (best_miou, miou, per_class_ious)
from datasets_custom.labels import GTA5Labels_TaskCV2017


# Configuration
num_classes = 19
context_path = 'resnet18'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 2
model_path = "/content/drive/MyDrive/MLDL_repo/step4/final4a.pth"
dataset_root = '/content/dataset/Cityscapes/Cityspaces'

# Initialize model
model = BiSeNet(num_classes=num_classes, context_path=context_path)

# Handle GPU
if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs")
    model = nn.DataParallel(model)

model = model.to(device)

# Load saved weights
state_dict = torch.load(model_path, map_location=device)
if isinstance(model, nn.DataParallel):
    model.module.load_state_dict(state_dict)
else:
    model.load_state_dict(state_dict)

model.eval()

# Load test dataset

test_dataset = CityScapes(root=dataset_root, split='val', transform=get_cityscape_transform())
print(f"Test dataset size: {len(test_dataset)}")

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Validate
with torch.no_grad():
    best_miou, miou, per_class_ious = validate(model, test_loader, num_classes, device, best_miou=0.0)

    # results
    print("\n Validation Results on Cityscape:")
    print(f" - mIoU: {miou:.4f}")
    for idx, label in enumerate(GTA5Labels_TaskCV2017.list_):
        print(f"{label.name:>15}: IoU = {per_class_ious[idx]:.4f}")


Loaded 500 images for split: val
Test dataset size: 500


Validating: 100%|██████████| 250/250 [00:27<00:00,  9.12it/s]

 Validation mIoU: 0.2967
 New best mIoU found!

 Validation Results on Cityscape:
 - mIoU: 0.2967
           road: IoU = 0.8674
       sidewalk: IoU = 0.2468
       building: IoU = 0.7910
           wall: IoU = 0.2450
          fence: IoU = 0.1607
           pole: IoU = 0.0001
          light: IoU = 0.0101
           sign: IoU = 0.0019
     vegetation: IoU = 0.8005
        terrain: IoU = 0.2483
            sky: IoU = 0.8278
         person: IoU = 0.4056
          rider: IoU = 0.0567
            car: IoU = 0.7221
          truck: IoU = 0.1508
            bus: IoU = 0.0335
          train: IoU = 0.0142
      motocycle: IoU = 0.0498
        bicycle: IoU = 0.0040
